In [2]:
import os
import sys



In [3]:
%pwd

'c:\\Users\\bilal\\Documents\\mlops\\Wine-Quality\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\bilal\\Documents\\mlops\\Wine-Quality'

In [15]:
import urllib.request as request
from src.WineQuality import logger
import zipfile

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataInestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [17]:
from src.WineQuality.constants import *
from src.WineQuality.utils.common import read_yaml ,create_directories

In [24]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataInestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataInestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [27]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataInestionConfig):
        self.config=config

    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [28]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-10 19:02:03,482: INFO:wine-quality:27: YAML file config\config.yaml loaded successfully.]
[2025-05-10 19:02:03,484: INFO:wine-quality:27: YAML file params.yaml loaded successfully.]
[2025-05-10 19:02:03,484: INFO:wine-quality:27: YAML file schema.yaml loaded successfully.]
[2025-05-10 19:02:03,484: INFO:wine-quality:48: Creating directory: artifacts]
[2025-05-10 19:02:03,484: INFO:wine-quality:48: Creating directory: artifacts/data_ingestion]
[2025-05-10 19:02:04,134: INFO:wine-quality:14: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B41B:29FB6F:107A0C:128461:681